In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.maximize_window()

In [ ]:
URL = "https://music.apple.com/kw/playlist/top-100-india/pl.c0e98d2423e54c39b3df955c24df3cc5"

In [ ]:
driver.get(URL)

In [ ]:
allSongs = driver.find_elements_by_class_name("songs-list-row")

In [ ]:
len(allSongs)

In [ ]:
import collections

In [ ]:
coll = collections.defaultdict(dict)
for index, song in enumerate(allSongs):
    songRank = song.find_element_by_class_name("songs-list-row__rank")
    coll[index]['Rank'] = songRank.text
    songName = song.find_element_by_class_name("songs-list-row__song-name")
    coll[index]['Song'] = songName.text
    
    songArtists = []
    
    allArtists = song.find_element_by_class_name("songs-list__col--secondary")
    allArtists = allArtists.find_elements_by_tag_name("a")
    for artist in allArtists:
        songArtists.append(artist.text)
    
    coll[index]['Artist(s)'] = ','.join(songArtists)

    songAlbum = song.find_element_by_class_name("songs-list__col--tertiary")
    coll[index]['Album'] = songAlbum.text

In [ ]:
coll

In [ ]:
import pandas as pd
import json

In [ ]:
jsonData = json.dumps(coll)

In [ ]:
df = pd.read_json(jsonData)

In [ ]:
df = df.T

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine("sqlite:///song.db",echo=False)

In [ ]:
df.to_sql('songs',con=engine,index=False,index_label='Rank',if_exists='replace')

In [ ]:
engine.url

In [ ]:
engine.execute('select * from songs').fetchall()

In [1]:
import pandas as pd

In [17]:
df = pd.read_csv("./database/songs.csv")

In [18]:
df.head()

,Rank,Song,Artist(s),Album,GYD_TIMESTAMP
0,1,"Apna Bana Le (From ""Bhediya"")","Arijit Singh,Sachin-Jigar,Amitabh Bhattacharya","Apna Bana Le (From ""Bhediya"") - Single",2023-02-12 11:37:34.927516
1,2,52 Bars,"Karan Aujla,Ikky",Four You - EP,2023-02-12 11:37:34.927516
2,3,"Besharam Rang (From ""Pathaan"")","Vishal & Shekhar,Shilpa Rao,Caralisa Monteiro,...","Besharam Rang (From ""Pathaan"") - Single",2023-02-12 11:37:34.927516
3,4,Maan Meri Jaan,King,Champagne Talk,2023-02-12 11:37:34.927516
4,5,Jhoome Jo Pathaan,"Vishal & Shekhar,Arijit Singh,Sukriti Kakar,Vi...",Pathaan (Original Motion Picture Soundtrack),2023-02-12 11:37:34.927516


In [19]:
df = df.assign(artists=df['Artist(s)'].str.split(',')).explode('artists')
df['Artist(s)'] = df['artists']
df = df.drop("artists",axis=1)

In [21]:
df['Artist(s)'] = df['artists']

In [24]:
df = df.drop("artists",axis=1)

,Rank,Song,Artist(s),Album,GYD_TIMESTAMP
0,1,"Apna Bana Le (From ""Bhediya"")",Arijit Singh,"Apna Bana Le (From ""Bhediya"") - Single",2023-02-12 11:37:34.927516
0,1,"Apna Bana Le (From ""Bhediya"")",Sachin-Jigar,"Apna Bana Le (From ""Bhediya"") - Single",2023-02-12 11:37:34.927516
0,1,"Apna Bana Le (From ""Bhediya"")",Amitabh Bhattacharya,"Apna Bana Le (From ""Bhediya"") - Single",2023-02-12 11:37:34.927516
1,2,52 Bars,Karan Aujla,Four You - EP,2023-02-12 11:37:34.927516
1,2,52 Bars,Ikky,Four You - EP,2023-02-12 11:37:34.927516
...,...,...,...,...,...
97,98,Tera Ban Jaunga,Tulsi Kumar,Kabir Singh (Original Motion Picture Soundtrack),2023-02-12 11:37:34.927516
98,99,Desires,AP Dhillon,Hidden Gems - EP,2023-02-12 11:37:34.927516
98,99,Desires,Gurinder Gill,Hidden Gems - EP,2023-02-12 11:37:34.927516
99,100,Dil Diyan Gallan,Vishal & Shekhar,Tiger Zinda Hai (Original Motion Picture Sound...,2023-02-12 11:37:34.927516


In [ ]:
# split the "artists" column into a list of values
df['artists'] = df['Artist(s)'].str.split(', ')

# use the melt method to create a new row for each value in the "artists" column
df_expanded = pd.melt(df, id_vars='song', value_vars='artists')

# drop the original "artists" column and keep only the values from the "value" column
df_expanded = df_expanded.drop('variable', axis=1)
df_expanded = df_expanded.rename(columns={'value': 'artists'})